**LOADING DATASET**

In [4]:
from nltk.tag.stanford import StanfordPOSTagger
from nltk.stem.snowball import SnowballStemmer

jar = "/content/drive/MyDrive/Dataset Project/stanford-postagger.jar"
model = "/content/drive/MyDrive/Dataset Project/english-bidirectional-distsim.tagger"
pos = StanfordPOSTagger(model, jar, encoding = "utf-8")
stemmer = SnowballStemmer("english")

/usr/local/lib/python3.7/dist-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [5]:
#start
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'snli', split='train')

     |████████████████████████████████| 290 kB 12.5 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 1.1 MB 73.0 MB/s 
     |████████████████████████████████| 132 kB 75.4 MB/s 
     |████████████████████████████████| 243 kB 72.7 MB/s 
     |████████████████████████████████| 192 kB 72.2 MB/s 
     |████████████████████████████████| 160 kB 72.9 MB/s 
     |████████████████████████████████| 271 kB 75.2 MB/s 


Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/938 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd

df_train = pd.DataFrame(list(zip(dataset['premise'], dataset['hypothesis'], dataset['label'])), 
                        columns =['premise', 'hypothesis', 'label'])
df_train

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,1
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",0
3,Children smiling and waving at camera,They are smiling at their parents,1
4,Children smiling and waving at camera,There are children present,0
...,...,...,...
550147,Four dirty and barefooted children.,four kids won awards for 'cleanest feet',2
550148,Four dirty and barefooted children.,"four homeless children had their shoes stolen,...",1
550149,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is competing in a surfing ...,1
550150,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...,2


**EXTRACTING DATA FROM DATASET**

In [8]:
df = df_train.drop_duplicates(subset=['premise'])
df = df[75000:]
df.reset_index(drop=True, inplace=True)

In [9]:
df

,premise,hypothesis,label
0,A guy jumping off a dock into a body of water.,A man is jumping of a dock into a big yellow i...,1
1,Two men in wetsuits are surfing along the edge...,Two women surf on a large wave.,2
2,A group of people are conversing with one anot...,a group of people talking to each other,0
3,A black dog runs through a muddy bank.,A dog covered in mud.,0
4,Two dogs are being led into a building on leas...,An owner and his dogs are going inside from a ...,1
...,...,...,...
75731,A group of four kids stand in front of a statu...,the kids are black,1
75732,a kid doing tricks on a skateboard on a bridge,a kid is skateboarding,0
75733,A dog with a blue collar plays ball outside.,a dog is on the couch,2
75734,Four dirty and barefooted children.,four children have dirty feet.,0


**GET ORIGINAL POS-TAGS OF THE SENTENCE**

In [10]:
def getOrigWordsTags(POS):
  words = []
  tags = []
  for word, tag in POS:
    words.append(word)
    tags.append(tag)
  return words,tags

**GENERATE POSSIBLE GENERAL QUESTIONS ON A STATEMENT**

In [11]:
def getGenQuestions(sentence, ans='Yes'):
  q = []
  a = []
  pos_tagged = pos.tag(sentence.split())
  
  words, tags = getOrigWordsTags(pos_tagged)
  list_vbz_swap = ['is','was','am', 'are','will','would','shall','should','can','could','has','have','may','might', 'there' ]

  if ('PRP' in tags or 'NNP' in tags or 'DT' in tags or 'EX' in tags):
    p2 = None
    for word in words:
      if word in list_vbz_swap:
        p2 = words.index(word)
        break
    if p2 != None:
      if 'PRP' in tags:
        p1 = tags.index('PRP')
      elif 'NNP' in tags:
        p1 = tags.index('NNP')
      elif 'EX' in tags:
        p1 = tags.index('EX')
      else: 
        p1 = tags.index('DT')
      tempw = words[p1]
      tempt = tags[p1]
      words[p1] = words[p2]
      words[p2] = tempw
      tags[p1] = tags[p2]
      tags[p2] = tempt
      w = words
      question = ' '.join(words) + '?'
      if question not in q:
        q.append(question)
        a.append(ans)
  
  words, tags = getOrigWordsTags(pos_tagged)

  for idx, tag in enumerate(tags):
    if  tag == 'VBD'and words[idx] not in list_vbz_swap:
      words[idx] = stemmer.stem(words[idx])
      question = "Did ".join(words) + '?'
      if question not in q:
        q.append(question)  
        a.append(ans)
      break
    if  tag == 'VBZ'and words[idx] not in list_vbz_swap:
      words[idx] = stemmer.stem(words[idx])
      question = "Does " + ' '.join(words) + '?'
      if question not in q:
        q.append(question) 
        a.append(ans) 
      break
    if tag == "VBP" and words[idx] not in list_vbz_swap:
      words[idx] = stemmer.stem(words[idx])
      question = "Do " + ' '.join(words) + '?'
      if question not in q:
        q.append(question)
        a.append(ans)  
      break

  if 'VBG' in tags:
    if 'NNS' in tags:
      question = 'Are ' + ' '.join(words) + '?'
      if question not in q:
        q.append(question)
        a.append(ans)
      else:
        if 'is' in p_words:
          words.remove('is')
          question = 'Is ' + ' '.join(words) + '?'
          if question not in q:
            q.append(question)
            a.append(ans)
  return q,a

**GET ENTAILMENT FORM OF GIVEN SET OF STATEMENTS**

In [12]:
def getEntailment(rule_name, p, q, r='') :
  entail = ''
  p_postag = pos.tag(p.split())
  q_postag = pos.tag(q.split())
  p_words, p_tags = getOrigWordsTags(p_postag)
  q_words, q_tags = getOrigWordsTags(q_postag)

  if rule_name.lower() == 'conjunction':
    entail = p + " and " + q
  elif rule_name.lower() == 'addition':
    entail = p + " or " + q
  elif r != '' and rule_name.lower() == 'composition':
    entail = p + " implies " + q + " and " + r
  elif rule_name.lower() == 'de morgans1':
    list_vbz_swap = ['is','was','am', 'are','will','would','shall','should','can','could','has','have','may','might']
    not_p = ''
    not_q = ''
    for id in range(0, len(p_words)):
      if (p_words[id] in list_vbz_swap):
        not_p = ' '.join(p_words[:id+1]) + " not " + ' '.join(p_words[id+1:])
    for id in range(0, len(q_words)):
      if (q_words[id] in list_vbz_swap):
        not_q = ' '.join(q_words[:id+1]) + " not " + ' '.join(q_words[id+1:])
    entail = "Either " + not_p + " or " + not_q
    if entail == None:
      entail = "Either not " + p + " or not " + q
  return entail

**GENERATING CONJUNCTION SAMPLES**



In [ ]:
#writing sentences, questions and answers for Conjunction

sentences = []
questions = []
answers = []
rule = []
sample = df[:200]
i=0
for row in sample.iterrows():
  if(i<99):
    p = sample['premise'][i].replace(".","")
    q = sample['premise'][i+1]
    entail = getEntailment('Conjunction',p,q)

    p_postag = pos.tag(p.split())
    q_postag = pos.tag(q.split())
    entail_postag = pos.tag(entail.split())
    p_words, p_tags = getOrigWordsTags(p_postag)
    q_words, q_tags = getOrigWordsTags(q_postag)
    entail_words, entail_tags = getOrigWordsTags(entail_postag)

    list_vbz_swap = ['is','was','am', 'are','will','would','shall','should','can','could','has','have','may','might']
    not_p = ''
    not_q = ''
    for id in range(0, len(p_words)):
      if (p_words[id] in list_vbz_swap):
        not_p = ' '.join(p_words[:id+1]) + " not " + ' '.join(p_words[id+1:])
    for id in range(0, len(q_words)):
      if (q_words[id] in list_vbz_swap):
        not_q = ' '.join(q_words[:id+1]) + " not " + ' '.join(q_words[id+1:])

    qs1, ans1 = getGenQuestions(p, 'Yes')
    qs2, ans2 = getGenQuestions(q, 'Yes')
    qs3, ans3 = getGenQuestions(entail, 'Yes')
    qs = qs1 + qs2 + qs3
    ans = ans1 + ans2 + ans3 
    if not_p != '':
      qs4, ans4 = getGenQuestions(not_p, 'No')
      qs += qs4
      ans += ans4
    if not_q != '':
      qs5, ans5 = getGenQuestions(not_q, 'No')
      qs += qs5
      ans += ans5

    for j in range(0,len(qs)):
      sentences.append(p + '. ' + q)
      questions.append(qs[j])
      answers.append(ans[j])
      rule.append("p,q |- ( p ^ q )")

    question = "Is " + p + " and " + q + " true?"
    questions.append(question)
    sentences.append(p + '. ' + q)
    answers.append('Yes')
    rule.append("p,q |- ( p ^ q )")
    question = "Is " + p + " and " + q + " not true?"
    questions.append(question)
    sentences.append(p + '. ' + q)
    answers.append('No')
    rule.append("p,q |- ( p ^ q )")
  i=i+2

In [ ]:
df_co1 = pd.DataFrame(list(rule))
df_co2 = pd.DataFrame(list(sentences)) 
df_co3 = pd.DataFrame(list(questions))
df_co4 = pd.DataFrame(list(answers))
df_co = [df_co1, df_co2, df_co3, df_co4]
df_conjunction = pd.concat(df_co, axis = 1)
df_conjunction.columns = ['Rule','Context', 'Question', 'Answer']
df_conjunction.head()

,Rule,Context,Question,Answer
0,"p,q |- ( p ^ q )",A guy jumping off a dock into a body of water....,Two men in wetsuits the surfing along are edge...,Yes
1,"p,q |- ( p ^ q )",A guy jumping off a dock into a body of water....,Are Two men in wetsuits are surfing along the ...,Yes
2,"p,q |- ( p ^ q )",A guy jumping off a dock into a body of water....,are guy jumping off a dock into a body of wate...,Yes
3,"p,q |- ( p ^ q )",A guy jumping off a dock into a body of water....,Are A guy jumping off a dock into a body of wa...,Yes
4,"p,q |- ( p ^ q )",A guy jumping off a dock into a body of water....,Two men in wetsuits the not surfing along are ...,No


In [ ]:
df_conjunction.to_csv('/content/drive/MyDrive/Dataset Project/conjunction.csv')

**GENERATING ADDITION SAMPLES**

In [ ]:
#writing sentences, questions and answers for Addition

sentences = []
questions = []
answers = []
rule = []
sample = df[200:400]
i=200
for row in sample.iterrows():
  if(i<299):
    p = sample['premise'][i].replace(".","")
    q = sample['premise'][i+1]
    entail = getEntailment('Addition',p,q)

    p_postag = pos.tag(p.split())
    q_postag = pos.tag(q.split())
    entail_postag = pos.tag(entail.split())
    p_words, p_tags = getOrigWordsTags(p_postag)
    q_words, q_tags = getOrigWordsTags(q_postag)
    entail_words, entail_tags = getOrigWordsTags(entail_postag)

    list_vbz_swap = ['is','was','am', 'are','will','would','shall','should','can','could','has','have','may','might']
    not_p = ''
    not_q = ''
    for id in range(0, len(p_words)):
      if (p_words[id] in list_vbz_swap):
        not_p = ' '.join(p_words[:id+1]) + " not " + ' '.join(p_words[id+1:])
    for id in range(0, len(q_words)):
      if (q_words[id] in list_vbz_swap):
        not_q = ' '.join(q_words[:id+1]) + " not " + ' '.join(q_words[id+1:])

    qs1, ans1 = getGenQuestions(p, 'Yes')
    qs2, ans2 = getGenQuestions(q, 'Yes')
    qs2, ans3 = getGenQuestions(entail, 'Yes')
    qs = qs1 + qs2 + qs3
    ans = ans1 + ans2 + ans3 
    if not_p != '':
      qs4, ans4 = getGenQuestions(not_p, 'No')
      qs += qs4
      ans += ans4
    if not_q != '':
      qs5, ans5 = getGenQuestions(not_q, 'No')
      qs += qs5
      ans += ans5

    for j in range(0,len(ans)-1):
      sentences.append(p + '. ' + q)
      questions.append(qs[j])
      answers.append(ans[j])
      rule.append("p|- ( p v q )")
      
    question = "Is either " + p + " or " + q + " true?"
    questions.append(question)
    sentences.append(p + '. ' + q)
    answers.append('Yes')
    rule.append("p|- ( p v q )")
    question = "If not " + p + ", is " + q + " true?"
    questions.append(question)
    sentences.append(p + '. ' + q)
    answers.append('No')
    rule.append("p|- ( p v q )")
    if not_q != '':
      question = "Is " + p + " and " + not_q + " true?"
      questions.append(question)
      sentences.append(p + '. ' + q)
      answers.append('Yes')
      rule.append("p|- ( p v q )")
  i=i+2

In [ ]:
df_add1 = pd.DataFrame(list(rule))
df_add2 = pd.DataFrame(list(sentences)) 
df_add3 = pd.DataFrame(list(questions))
df_add4 = pd.DataFrame(list(answers))
df_add = [df_add1, df_add2, df_add3, df_add4]
df_addition = pd.concat(df_add, axis = 1)
df_addition.columns = ['Rule','Context', 'Question', 'Answer']
df_addition.head()

,Rule,Context,Question,Answer
0,p|- ( p v q ),A man in black is eating a baked good near two...,is man in black A eating a baked good near two...,Yes
1,p|- ( p v q ),A man in black is eating a baked good near two...,Are A man in black is eating a baked good near...,Yes
2,p|- ( p v q ),A man in black is eating a baked good near two...,is man in black A eating a baked good near two...,Yes
3,p|- ( p v q ),A man in black is eating a baked good near two...,Are A man in black is eating a baked good near...,Yes
4,p|- ( p v q ),A man in black is eating a baked good near two...,are brown dog is sniffing a white dog in front...,No


In [ ]:
df_addition.to_csv('/content/drive/MyDrive/Dataset Project/addition.csv')

**GENERATING DE-MORGAN THEOREM (1)'S SAMPLES**

In [ ]:
#writing sentences, questions and answers for De Morgan's (1)
sentences = []
questions = []
answers = []
rule = []
sample = df[400:600]
i=400
for row in sample.iterrows():
  if(i<499):
    p = sample['premise'][i].replace(".","")
    q = sample['premise'][i+1]
    entail = getEntailment('De Morgans1',p,q)

    p_postag = pos.tag(p.split())
    q_postag = pos.tag(q.split())
    entail_postag = pos.tag(entail.split())
    p_words, p_tags = getOrigWordsTags(p_postag)
    q_words, q_tags = getOrigWordsTags(q_postag)
    entail_words, entail_tags = getOrigWordsTags(entail_postag)

    list_vbz_swap = ['is','was','am', 'are','will','would','shall','should','can','could','has','have','may','might']
    not_p = ''
    not_q = ''
    for id in range(0, len(p_words)):
      if (p_words[id] in list_vbz_swap):
        not_p = ' '.join(p_words[:id+1]) + " not " + ' '.join(p_words[id+1:])
    for id in range(0, len(q_words)):
      if (q_words[id] in list_vbz_swap):
        not_q = ' '.join(q_words[:id+1]) + " not " + ' '.join(q_words[id+1:])

    qs1, ans1 = getGenQuestions(p, 'Yes')
    qs2, ans2 = getGenQuestions(q, 'Yes')
    qs3, ans3 = getGenQuestions(entail, 'Yes')
    qs = qs1 + qs2 + qs3
    ans = ans1 + ans2 + ans3 
    if not_p != '':
      qs4, ans4 = getGenQuestions(not_p, 'No')
      qs += qs4
      ans += ans4
    if not_q != '':
      qs5, ans5 = getGenQuestions(not_q, 'No')
      qs += qs5
      ans += ans5

    for j in range(0,len(qs)):
      sentences.append("Not " + p + ' and ' + q)
      questions.append(qs[j])
      answers.append(ans[j])
      rule.append('~( p v q ) |- ~p v ~q')
  i=i+2

In [ ]:
df_de1 = pd.DataFrame(list(rule)) 
df_de2 = pd.DataFrame(list(sentences)) 
df_de3 = pd.DataFrame(list(questions))
df_de4 = pd.DataFrame(list(answers))
df_de = [df_de1, df_de2, df_de3, df_de4]
df_demorgan = pd.concat(df_de, axis = 1)
df_demorgan.columns = ['Rule','Context', 'Question', 'Answer']
df_demorgan.head()

,Rule,Context,Question,Answer
0,~( p v q ) |- ~p v ~q,"Not A girl dressed in white, sitting down, hol...","Are A girl dressed in white, sitting down, hol...",Yes
1,~( p v q ) |- ~p v ~q,"Not A girl dressed in white, sitting down, hol...",Does A man prepar to throw a stick intended fo...,Yes
2,~( p v q ) |- ~p v ~q,Not People strolling a city street at night an...,Are People strolling a city street at night?,Yes
3,~( p v q ) |- ~p v ~q,Not A man wearing a black shirt is outside wal...,is man wearing a black shirt A outside walking...,Yes
4,~( p v q ) |- ~p v ~q,Not A man wearing a black shirt is outside wal...,Are Three people wearing dark costumes and mak...,Yes


In [ ]:
df_demorgan.to_csv('/content/drive/MyDrive/Dataset Project/demorgan.csv')

**GENERATING COMPOSITION SAMPLES**

In [ ]:
#writing sentences, questions and answers for Composition
sentences = []
questions = []
answers = []
rule = []
sample = df[600:750]
i=600
for row in sample.iterrows():
  if(i<649):
    p = sample['premise'][i].replace(".","")
    q = sample['premise'][i+1]
    r = sample['premise'][i+2]
    entail = getEntailment('Composition',p,q,r)

    p_postag = pos.tag(p.split())
    q_postag = pos.tag(q.split())
    r_postag = pos.tag(r.split())
    entail_postag = pos.tag(entail.split())
    p_words, p_tags = getOrigWordsTags(p_postag)
    q_words, q_tags = getOrigWordsTags(q_postag)
    r_words, r_tags = getOrigWordsTags(r_postag)
    entail_words, entail_tags = getOrigWordsTags(entail_postag)

    list_vbz_swap = ['is','was','am', 'are','will','would','shall','should','can','could','has','have','may','might']
    not_p = ''
    not_q = ''
    not_r = ''
    for id in range(0, len(p_words)):
      if (p_words[id] in list_vbz_swap):
        not_p = ' '.join(p_words[:id+1]) + " not " + ' '.join(p_words[id+1:])
    for id in range(0, len(q_words)):
      if (q_words[id] in list_vbz_swap):
        not_q = ' '.join(q_words[:id+1]) + " not " + ' '.join(q_words[id+1:])
    for id in range(0, len(r_words)):
      if (r_words[id] in list_vbz_swap):
        not_r = ' '.join(r_words[:id+1]) + " not " + ' '.join(r_words[id+1:])

    qs1, ans1 = getGenQuestions(p, 'Yes')
    qs2, ans2 = getGenQuestions(q, 'Yes')
    qs3, ans3 = getGenQuestions(r, 'Yes')
    qs4, ans4 = getGenQuestions(entail, 'Yes')
    qs = qs1 + qs2 + qs3 + qs4
    ans = ans1 + ans2 + ans3 + ans4
    if not_p != '':
      qs5, ans5 = getGenQuestions(not_p, 'No')
      qs += qs5
      ans += ans5
    if not_q != '':
      qs6, ans6 = getGenQuestions(not_q, 'No')
      qs += qs6
      ans += ans6
    if not_r != '':
      qs7, ans7 = getGenQuestions(not_r, 'No')
      qs += qs7
      ans += ans7

    for j in range(0,len(qs)-1):
      sentences.append("If " + p + ' implies ' + q + " and " + p + " implies " + r)
      questions.append(qs[j])
      answers.append(ans[j])
      rule.append("(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r ))")
    
    question = "If " + p + " is true, is " + q + " and " + r + " true?"
    questions.append(question)
    sentences.append("If " + p + ' implies ' + q + " and " + p + " implies " + r)
    answers.append('Yes')
    rule.append("(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r ))")
    question = "If " + p + " is not true, is " + q + " and " + r + " true?"
    questions.append(question)
    sentences.append("If " + p + ' implies ' + q + " and " + p + " implies " + r)
    answers.append('No')
    rule.append("(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r ))")
    question = "Does " + p + " not imply " + q + " and " + r + " ?"
    questions.append(question)
    sentences.append("If " + p + ' implies ' + q + " and " + p + " implies " + r)
    answers.append('No')
    rule.append("(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r ))")
    question = "Does " + p + " imply " + q + " and " + r + " ?"
    questions.append(question)
    sentences.append("If " + p + ' implies ' + q + " and " + p + " implies " + r)
    answers.append('Yes')
    rule.append("(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r ))")

  i=i+3

In [ ]:
df_comp1 = pd.DataFrame(list(rule)) 
df_comp2 = pd.DataFrame(list(sentences)) 
df_comp3 = pd.DataFrame(list(questions))
df_comp4 = pd.DataFrame(list(answers))
df_comp = [df_comp1, df_comp2, df_comp3, df_comp4]
df_composition = pd.concat(df_comp, axis = 1)
df_composition.columns = ['Rule','Context', 'Question', 'Answer']
df_composition.head()

,Rule,Context,Question,Answer
0,(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r )),If A thin young woman in a green t-shirt cycle...,is little boy wearing a blue shirt A playing a...,Yes
1,(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r )),If A thin young woman in a green t-shirt cycle...,is small child in a blue shirt A standing in f...,Yes
2,(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r )),If A thin young woman in a green t-shirt cycle...,is thin young woman in a green t-shirt cycles ...,Yes
3,(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r )),If A thin young woman in a green t-shirt cycle...,Does A thin young woman in a green t-shirt cyc...,Yes
4,(( p -> q ) ^ ( p -> r )) |- ( p -> ( q ^ r )),If A thin young woman in a green t-shirt cycle...,Are A thin young woman in a green t-shirt cycl...,Yes


In [ ]:
df_composition.to_csv('/content/drive/MyDrive/Dataset Project/composition.csv')

**FIRST ORDER LOGIC - FOL**

In [68]:
df_all = df_train.loc[df_train["premise"].str.startswith("All")]
df_all = df_all.drop_duplicates(subset=['premise'])
df_all.reset_index(drop=True, inplace=True)
df_every = df_train.loc[df_train["premise"].str.startswith("Every")]
df_every = df_every.drop_duplicates(subset=['premise'])
df_every.reset_index(drop=True, inplace=True)
df_some = df_train.loc[df_train["premise"].str.startswith("Some")]
df_some = df_some.drop_duplicates(subset=['premise'])
df_some.reset_index(drop=True, inplace=True)
df_there = df_train.loc[df_train["premise"].str.startswith("There")]
df_there = df_there.drop_duplicates(subset=['premise'])
df_there.reset_index(drop=True, inplace=True)

In [69]:
print(len(df_all))
print(len(df_every))
print(len(df_some))
print(len(df_there))

18
17
780
1126


In [70]:
df_all = df_all[10:]
df_all.reset_index(drop=True, inplace=True)
df_every = df_every[9:]
df_every.reset_index(drop=True, inplace=True)
df_some = df_some[390:]
df_some.reset_index(drop=True, inplace=True)

In [71]:
print(len(df_all))
print(len(df_every))
print(len(df_some))
print(len(df_there))

8
8
390
1126


In [81]:
def getFOLQuestions(sentence, key, ans='Yes'):
  q = []
  a = []
  pos_tagged = pos.tag(sentence.split())
  
  words, tags = getOrigWordsTags(pos_tagged)
  if key.lower() == 'Some'.lower() or key.lower() == 'All'.lower():
    if 'are' in words:
      p1 = words.index('are')
      question = 'Are ' + ' '.join(words[:p1]) + " "+ ' '.join(words[p1+1:]) + '?'
      q.append(question)
      a.append(ans)
    else:
      question = 'Do ' + ' '.join(words) + '?'
      q.append(question)
      a.append(ans)
  elif key == 'Every'.lower() or key == 'There'.lower():
    if 'is' in words:
      p1 = words.index('is')
      question = 'Is ' + ' '.join(words[:p1]) + ' '.join(words[p1+1:]) + '?'
      q.append(question)
      a.append(ans)
    else:
      question = 'Does ' + ' '.join(words) + '?'
      q.append(question)
      a.append(ans)
  return q,a

In [82]:
#writing sentences, questions and answers for FOL statements

def FOLQAGeneration(sample, key):
  sentences = []
  questions = []
  answers = []
  rules = []
  i=0
  for row in sample.iterrows():
    if(i<(len(sample)/2)):
      p = sample['premise'][i].replace(".","")
      q = sample['premise'][i+1]
      entail1 = getEntailment('Conjunction',p,q)
      entail2 = getEntailment('Addition',p,q)
      entail3 = getEntailment('De Morgans1',p,q)

      p_postag = pos.tag(p.split())
      q_postag = pos.tag(q.split())
      entail1_postag = pos.tag(entail1.split())
      entail2_postag = pos.tag(entail2.split())
      entail3_postag = pos.tag(entail3.split())

      p_words, p_tags = getOrigWordsTags(p_postag)
      q_words, q_tags = getOrigWordsTags(q_postag)
      entail1_words, entail1_tags = getOrigWordsTags(entail1_postag)
      entail2_words, entail2_tags = getOrigWordsTags(entail2_postag)
      entail3_words, entail3_tags = getOrigWordsTags(entail3_postag)

      list_vbz_swap = ['is','was','am', 'are','will','would','shall','should','can','could','has','have','may','might']
      not_p = ''
      not_q = ''
      for id in range(0, len(p_words)):
        if (p_words[id] in list_vbz_swap):
          not_p = ' '.join(p_words[:id+1]) + " not " + ' '.join(p_words[id+1:])
      for id in range(0, len(q_words)):
        if (q_words[id] in list_vbz_swap):
          not_q = ' '.join(q_words[:id+1]) + " not " + ' '.join(q_words[id+1:])

      qs1, ans1 = getFOLQuestions(p, key, 'Yes')
      qs2, ans2 = getFOLQuestions(q, key, 'Yes')
      qs = qs1 + qs2
      ans = ans1 + ans2
      if not_p != '':
        qs3, ans3 = getGenQuestions(not_p, 'No')
        qs += qs3
        ans += ans3
      if not_q != '':
        qs4, ans4 = getGenQuestions(not_q, 'No')
        qs += qs4
        ans += ans4

      for j in range(0,len(qs)):
        sentences.append(p + '. ' + q)
        questions.append(qs[j])
        answers.append(ans[j])
        rules.append("p,q |- ( p ^ q )")

      #Conjunction
      question = "Is " + p + " and " + q + " true?"
      questions.append(question)
      sentences.append(p + '. ' + q)
      answers.append('Yes')
      rules.append("p,q |- ( p ^ q )")
      question = "Is " + p + " and " + q + " not true?"
      questions.append(question)
      sentences.append(p + '. ' + q)
      answers.append('No')
      rules.append("p,q |- ( p ^ q )")

      #Addition
        
      question = "Is either " + p + " or " + q + " true?"
      questions.append(question)
      sentences.append(p + '. ' + q)
      answers.append('Yes')
      rules.append("p|- ( p v q )")
      question = "If not " + p + ", is " + q + " true?"
      questions.append(question)
      sentences.append(p + '. ' + q)
      answers.append('No')
      rules.append("p|- ( p v q )")
      if not_q != '':
        question = "Is " + p + " and " + not_q + " true?"
        questions.append(question)
        sentences.append(p + '. ' + q)
        answers.append('Yes')
        rules.append("p|- ( p v q )")

    i=i+2
  return sentences, questions, answers, rules

In [50]:
sentences, questions, answers, rules = FOLQAGeneration(df_all, 'All')
df_all1 = pd.DataFrame(list(rules)) 
df_all2 = pd.DataFrame(list(sentences)) 
df_all3 = pd.DataFrame(list(questions))
df_all4 = pd.DataFrame(list(answers))
df_folall = [df_all1, df_all2, df_all3, df_all4]
df_fol_all = pd.concat(df_folall, axis = 1)
df_fol_all.columns = ['Rule','Context', 'Question', 'Answer']
df_fol_all.head()

,Rule,Context,Question,Answer
0,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,Are All people flying in the military plane?,Yes
1,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,Do All American man dressed up for the patriot...,Yes
2,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,are people All not flying in the military plane?,No
3,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,Are All people are not flying in the military ...,No
4,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,Is All people are flying in the military plane...,Yes


In [51]:
df_fol_all

,Rule,Context,Question,Answer
0,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,Are All people flying in the military plane?,Yes
1,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,Do All American man dressed up for the patriot...,Yes
2,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,are people All not flying in the military plane?,No
3,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,Are All people are not flying in the military ...,No
4,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,Is All people are flying in the military plane...,Yes
5,"p,q |- ( p ^ q )",All people are flying in the military plane. A...,Is All people are flying in the military plane...,No
6,p|- ( p v q ),All people are flying in the military plane. A...,Is either All people are flying in the militar...,Yes
7,p|- ( p v q ),All people are flying in the military plane. A...,If not All people are flying in the military p...,No
8,"p,q |- ( p ^ q )",All these people are enjoying a basement dance...,Are All these people enjoying a basement dance...,Yes
9,"p,q |- ( p ^ q )",All these people are enjoying a basement dance...,Are All dancing with full energy to the music ...,Yes


In [54]:
sentences, questions, answers, rules = FOLQAGeneration(df_every, 'Every')
df_every1 = pd.DataFrame(list(rules)) 
df_every2 = pd.DataFrame(list(sentences)) 
df_every3 = pd.DataFrame(list(questions))
df_every4 = pd.DataFrame(list(answers))
df_folevery = [df_every1, df_every2, df_every3, df_every4]
df_fol_every = pd.concat(df_folevery, axis = 1)
df_fol_every.columns = ['Rule','Context', 'Question', 'Answer']
df_fol_every.head()

,Rule,Context,Question,Answer
0,"p,q |- ( p ^ q )",Everyone is waiting in line outside the buildi...,Is Everyonewaiting in line outside the building?,Yes
1,"p,q |- ( p ^ q )",Everyone is waiting in line outside the buildi...,Is Everyoneclapping in this office.?,Yes
2,"p,q |- ( p ^ q )",Everyone is waiting in line outside the buildi...,Everyone the not waiting in line outside is bu...,No
3,"p,q |- ( p ^ q )",Everyone is waiting in line outside the buildi...,Everyone this not clapping in is office.?,No
4,"p,q |- ( p ^ q )",Everyone is waiting in line outside the buildi...,Is Everyone is waiting in line outside the bui...,Yes


In [55]:
sentences, questions, answers, rules = FOLQAGeneration(df_some, 'Some')
df_some1 = pd.DataFrame(list(rules)) 
df_some2 = pd.DataFrame(list(sentences)) 
df_some3 = pd.DataFrame(list(questions))
df_some4 = pd.DataFrame(list(answers))
df_folsome = [df_some1, df_some2, df_some3, df_some4]
df_fol_some = pd.concat(df_folsome, axis = 1)
df_fol_some.columns = ['Rule','Context', 'Question', 'Answer']
df_fol_some.head()

,Rule,Context,Question,Answer
0,"p,q |- ( p ^ q )","Some people sit, and some stand in a group of ...","Do Some people sit, and some stand in a group ...",Yes
1,"p,q |- ( p ^ q )","Some people sit, and some stand in a group of ...",Are Some people walking on a street looking at...,Yes
2,"p,q |- ( p ^ q )","Some people sit, and some stand in a group of ...",are people Some not walking on a street lookin...,No
3,"p,q |- ( p ^ q )","Some people sit, and some stand in a group of ...",Are Some people are not walking on a street lo...,No
4,"p,q |- ( p ^ q )","Some people sit, and some stand in a group of ...","Is Some people sit, and some stand in a group ...",Yes


In [83]:
sentences, questions, answers, rules = FOLQAGeneration(df_there[:300], 'there')
df_there1 = pd.DataFrame(list(rules)) 
df_there2 = pd.DataFrame(list(sentences)) 
df_there3 = pd.DataFrame(list(questions))
df_there4 = pd.DataFrame(list(answers))
df_folthere = [df_there1, df_there2, df_there3, df_there4]
df_fol_there = pd.concat(df_folthere, axis = 1)
df_fol_there.columns = ['Rule','Context', 'Question', 'Answer']
df_fol_there.head()

,Rule,Context,Question,Answer
0,"p,q |- ( p ^ q )","There is a woman holding a baby, along with a ...","Is Therea woman holding a baby, along with a m...",Yes
1,"p,q |- ( p ^ q )","There is a woman holding a baby, along with a ...",Does There are cars driving down a street with...,Yes
2,"p,q |- ( p ^ q )","There is a woman holding a baby, along with a ...","is There not a woman holding a baby, along wit...",No
3,"p,q |- ( p ^ q )","There is a woman holding a baby, along with a ...","Are There is not a woman holding a baby, along...",No
4,"p,q |- ( p ^ q )","There is a woman holding a baby, along with a ...",There Golden. not cars driving down a street w...,No


In [85]:
df_fol_all.to_csv('/content/drive/MyDrive/Dataset Project/fol_all.csv')
df_fol_every.to_csv('/content/drive/MyDrive/Dataset Project/fol_every.csv')
df_fol_some.to_csv('/content/drive/MyDrive/Dataset Project/fol_some.csv')
df_fol_there.to_csv('/content/drive/MyDrive/Dataset Project/fol_there.csv')

**CREATING A CSV OUTPUT FILE**

In [86]:
add = pd.read_csv("/content/drive/MyDrive/Dataset Project/addition.csv")
conj = pd.read_csv("/content/drive/MyDrive/Dataset Project/conjunction.csv")
comp = pd.read_csv("/content/drive/MyDrive/Dataset Project/composition.csv")
demo = pd.read_csv("/content/drive/MyDrive/Dataset Project/demorgan.csv")
all = pd.read_csv("/content/drive/MyDrive/Dataset Project/fol_all.csv")
every = pd.read_csv("/content/drive/MyDrive/Dataset Project/fol_every.csv")
some = pd.read_csv("/content/drive/MyDrive/Dataset Project/fol_some.csv")
there = pd.read_csv("/content/drive/MyDrive/Dataset Project/fol_there.csv")

In [88]:
result = [conj, add, demo, comp, all, every, some, there]
output = pd.concat(result)
output.to_csv('/content/drive/MyDrive/Dataset Project/Dataset5_MamathaJuluru.csv')